In [1]:
!pip install gradio
!pip install --upgrade music21

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 95.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 10.8 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [2]:
!add-apt-repository ppa:mscore-ubuntu/mscore-stable -y
!apt-get update
!apt-get install musescore

Repository: 'deb https://ppa.launchpadcontent.net/mscore-ubuntu/mscore-stable/ubuntu/ jammy main'
Description:
~~~~~~~~~ MUSESCORE 2 RELEASES FOR UBUNTU 12.04 AND LATER ~~~~~~~~~
Note that Canonical doesn’t allow new builds for releases marked obsolete
(any nōn-LTS predating the previous LTS (currently 24.04 “noble”), and any LTS
older than 7 years, currently anything before 14.04 “trusty”). The packages
for those older releases here can no longer be updated but still work, within
limitations. Best to upgrade to a supported release of the operating system!

INSTALLING: (run these commands from the terminal)
  sudo add-apt-repository ppa:mscore-ubuntu/mscore-stable
  sudo apt-get update
  sudo apt-get install musescore

DO *NOT* USE THESE PPAs on Debian, only on Ubuntu!

‣‣‣ These packages are NOT suitable for ”KDE neon 18.04 (bionic)”!
‣ Use https://build.opensuse.org/project/show/home:mirabile:mscore instead.
⚠ KDE neon 18.04 was discontinued so those packages can no longer be updated

In [3]:
!apt-get install xvfb

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libfontenc1 libxfont2 libxkbfile1 x11-xkb-utils xfonts-base xfonts-encodings xfonts-utils
  xserver-common
The following NEW packages will be installed:
  libfontenc1 libxfont2 libxkbfile1 x11-xkb-utils xfonts-base xfonts-encodings xfonts-utils
  xserver-common xvfb
0 upgraded, 9 newly installed, 0 to remove and 59 not upgraded.
Need to get 7,815 kB of archives.
After this operation, 11.9 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libfontenc1 amd64 1:1.1.4-1build3 [14.7 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libxfont2 amd64 1:2.0.5-1build1 [94.5 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 libxkbfile1 amd64 1:1.1.0-1build3 [71.8 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/main amd64 x11-xkb-utils amd64 7.7+5build4 [172 kB]
Get:5 http://archiv

In [4]:
import gradio as gr
from music21 import *

In [5]:
import gradio as gr
from music21 import *

# Function to parse music files and compare them
def parse_and_compare(score1_file, score2_file):
    try:
        # Parse the input music scores using the file paths
        parsedWork1 = converter.parse(score1_file.name)
        parsedWork2 = converter.parse(score2_file.name)

        recurse_work1 = parsedWork1.recurse()
        recurse_work2 = parsedWork2.recurse()

        # Create empty score objects
        new_score1 = stream.Score()
        new_score2 = stream.Score()

        # Append notes and rests to the score objects
        for element in recurse_work1.notesAndRests:
            new_score1.append(element)

        for element in recurse_work2.notesAndRests:
            new_score2.append(element)

        # Lists to track skipped and extra notes along with their measures
        skipped_notes = []  # Notes removed from score1
        extra_notes = []    # Notes removed from score2
        correct_notes = 0   # Count of correct notes
        total_notes = 0     # Count of total notes

        # Mistakes counter to track types of mistakes
        mistakes_counter = {
            'note_mismatch': 0,  # Incorrect note played
            'rhythm_mismatch': 0,  # Incorrect rhythm
            'chord_mismatch': 0,  # Incorrect chord
            'missed_note': 0,  # Missed note (rest instead)
            'extra_note': 0,  # Extra note (played during rest)
        }

        # Function to handle note value discrepancies
        def handle_note_value_discrepancies(score1, score2, index):
            total_deleted_length = 0
            while index < len(score1) and index < len(score2):
                note1 = score1[index]
                note2 = score2[index]

                if note1.quarterLength != note2.quarterLength:
                    if note1.quarterLength > note2.quarterLength:
                        # Remove notes from score2 (extra notes)
                        while total_deleted_length < (note1.quarterLength - note2.quarterLength):
                            if index < len(score2):
                                # Capture the correct note and its measure for removal
                                measure = note2.getContextByClass(stream.Measure)
                                extra_notes.append((note2, measure))  # Track extra notes with measure
                                score2.pop(index)  # Remove the note from score2
                                total_deleted_length += note2.quarterLength
                            else:
                                break
                    else:
                        # Remove notes from score1 (skipped notes)
                        while total_deleted_length < (note2.quarterLength - note1.quarterLength):
                            if index < len(score1):
                                # Capture the correct note and its measure for removal
                                measure = note1.getContextByClass(stream.Measure)
                                skipped_notes.append((note1, measure))  # Track skipped notes with measure
                                score1.pop(index)  # Remove the note from score1
                                total_deleted_length += note1.quarterLength
                            else:
                                break
                else:
                    # No discrepancy, break the loop
                    break

                return index  # This should return the index when modifications happen.

            return index + 1  # We increment index only when no changes happen, and the next note needs to be processed.

        # Dictionary to store feedback by measure number
        feedback_group = {}

        i = 0
        while i < min(len(new_score1), len(new_score2)):
            new_i = handle_note_value_discrepancies(new_score1, new_score2, i)
            if new_i is not None:
                i = new_i
                if i < len(new_score1) and i < len(new_score2):
                    note1 = new_score1[i]
                    note2 = new_score2[i]
                    measure = note1.getContextByClass(stream.Measure)

                    # Initialize measure feedback if it doesn't exist
                    if measure is not None:
                        if measure.measureNumber not in feedback_group:
                            feedback_group[measure.measureNumber] = []

            i += 1

        # Comparing note counts
        if len(new_score1) != len(new_score2):
            feedback_group['Note Count'] = [f"Different amount of notes: {len(new_score1)} vs {len(new_score2)}"]
        else:
            feedback_group['Note Count'] = [f"Note Count: {len(new_score1)} vs {len(new_score2)}"]


        # Iterate through the scores and provide feedback on musical accuracy
        for i in range(min(len(new_score1), len(new_score2))):
            note1 = new_score1[i]
            note2 = new_score2[i]
            measure = note1.getContextByClass(stream.Measure)

            if measure is not None:
                if measure.measureNumber not in feedback_group:
                    feedback_group[measure.measureNumber] = []

            # Check various conditions and append feedback to the correct measure
            if note1.isRest and note2.isRest:
                correct_notes += 1
                total_notes += 1
                continue  # Skip comparison if both are rests
            elif note1.isRest and not note2.isRest:
                feedback_group[measure.measureNumber].append(f"⚠️ You played a note during a rest: rest vs What you played - {note2.nameWithOctave}")
                mistakes_counter['extra_note'] += 1  # Track extra note mistake
                total_notes += 1
            elif note2.isRest and not note1.isRest:
                feedback_group[measure.measureNumber].append(f"❌ You missed a note and added a rest: Rest vs Correct note - {note1.nameWithOctave}")
                mistakes_counter['missed_note'] += 1  # Track missed note mistake
                total_notes += 1
            else:
                # Compare chords
                if note1.isChord and note2.isChord:
                    played_chord1 = [p.name for p in note1.pitches]
                    played_chord2 = [p.name for p in note2.pitches]

                    if set(played_chord1) != set(played_chord2):
                        feedback_group[measure.measureNumber].append(f"⚠️ Wrong chord played: You played ({', '.join(played_chord1)}), the correct chord is ({', '.join(played_chord2)})")
                        mistakes_counter['chord_mismatch'] += 1  # Track chord mismatch
                    else:
                        correct_notes += 1
                        total_notes += 1
                elif note1.isChord and not note2.isChord:
                    # If note1 is a chord and note2 is not, still need to define played_chord2
                    played_chord2 = [p.name for p in note1.pitches]  # Use note1 for the correct chord
                    feedback_group[measure.measureNumber].append(f"❌ You did not play the correct chord. The correct chord is ({', '.join(played_chord2)})")
                    mistakes_counter['chord_mismatch'] += 1  # Track chord mismatch
                    total_notes += 1
                elif not note1.isChord and note2.isChord:
                    feedback_group[measure.measureNumber].append(f"⚠️ A note was incorrectly played as a chord")
                    mistakes_counter['chord_mismatch'] += 1  # Track chord mismatch
                # Check for both note and rhythm mismatches
                elif note1.nameWithOctave != note2.nameWithOctave and note1.quarterLength != note2.quarterLength:
                    correct_duration_name = note1.duration.type
                    played_duration_name = note2.duration.type
                    feedback_group[measure.measureNumber].append(f"❌ Wrong note and wrong rhythm: Correct note: {note1.nameWithOctave} ({correct_duration_name}) vs What you played: {note2.nameWithOctave} ({played_duration_name})")
                    mistakes_counter['note_mismatch'] += 1  # Track note mismatch
                    total_notes += 1
                # Check for rhythm mismatches
                elif note1.quarterLength != note2.quarterLength and note1.nameWithOctave == note2.nameWithOctave:
                    correct_duration_name = note1.duration.type
                    played_duration_name = note2.duration.type
                    feedback_group[measure.measureNumber].append(f"⚠️ Wrong rhythm: Correct note value: {correct_duration_name} vs What you played: {played_duration_name}")
                    mistakes_counter['rhythm_mismatch'] += 1  # Track rhythm mismatch
                    total_notes += 1
                # Check for note mismatches
                elif note1.quarterLength == note2.quarterLength and note1.nameWithOctave != note2.nameWithOctave:
                    feedback_group[measure.measureNumber].append(f"❌ Incorrect note played: Correct note: {note1.nameWithOctave} vs What you played: {note2.nameWithOctave}")
                    mistakes_counter['note_mismatch'] += 1  # Track note mismatch
                    total_notes += 1
                else:
                    correct_notes += 1
                    total_notes += 1

        # Calculate accuracy percentage
        accuracy_percentage = (correct_notes / total_notes) * 100 if total_notes > 0 else 100

        # Format feedback output, showing only measures with errors
        feedback_output = f"Your accuracy: {accuracy_percentage:.2f}%\n\n"
        for measure, feedback in feedback_group.items():
            if measure != 'Note Count' and feedback:  # Only show measures that have feedback
                feedback_output += f"Measure {measure}:\n" + "\n".join(feedback) + "\n\n"

        # Provide the feedback along with skipped and extra notes
        skipped_notes_text = f"Notes you may have skipped: {len(skipped_notes)}\n"
        skipped_notes_text += "\n".join([f"  - {note[0].nameWithOctave} in measure {note[1].measureNumber}" for note in skipped_notes]) + "\n"

        extra_notes_text = f"Extra notes you may have played: {len(extra_notes)}\n"
        extra_notes_text += "\n".join([f"  - {note[0].nameWithOctave} in measure {note[1].measureNumber}" for note in extra_notes]) + "\n"

        # Recommendation for improvement based on mistake types
        recommendations = []
        if mistakes_counter['note_mismatch'] > mistakes_counter['rhythm_mismatch']:
            recommendations.append("Focus on playing the correct notes first, then pay attention to rhythm.")
        elif mistakes_counter['rhythm_mismatch'] > mistakes_counter['note_mismatch']:
            recommendations.append("Your rhythm needs improvement. Focus on getting the correct timing first.")
        if mistakes_counter['chord_mismatch'] > 0:
            recommendations.append("Review your chord shapes and their inversions.")
        if mistakes_counter['missed_note'] > 0:
            recommendations.append("Make sure you don't miss any notes, especially if it's a rest you need to replace.")
        if mistakes_counter['extra_note'] > 0:
            recommendations.append("Avoid playing extra notes during rests.")

        recommendations_text = "\n".join(recommendations) if recommendations else "You're doing well! Keep practicing."

        return feedback_output, skipped_notes_text, extra_notes_text, recommendations_text

    except Exception as e:
        return f"Error occurred: {str(e)}", "", "", ""

# Gradio interface function
def get_feedback(score1_file, score2_file):
    return parse_and_compare(score1_file, score2_file)

# Set up Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("### Music Feedback")
    with gr.Row():
        score1_file = gr.File(label="Upload Reference Score")
        score2_file = gr.File(label="Upload Student's Score")
        submit_button = gr.Button("Submit")
    output_text = gr.Textbox(label="Feedback", lines=20)
    recommendations_output = gr.Textbox(label="Recommendations", lines=5)
    skipped_notes_output = gr.Textbox(label="Skipped Notes", lines=5)
    extra_notes_output = gr.Textbox(label="Extra Notes", lines=5)


    submit_button.click(get_feedback, inputs=[score1_file, score2_file], outputs=[output_text, skipped_notes_output, extra_notes_output, recommendations_output])

demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4bb262191669fd14fe.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
